In [1]:
import os
import groq
import json
from groq import Groq

In [2]:
first_tools = [
        # Tool 1 - Get Exchange Rate
        { 
            "type": "function",
            "function": {
                "name": "get_exchange_rate",
                "description": "Get the current exchange rate of a base currency and target currency",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "base_currency": {
                            "type": "string",
                            "description": "The base currency for exchange rate calculations, i.e. USD, EUR, RUB",
                        },
                        "target_currency": {
                            "type": "string", 
                            "description": "The target currency for exchange rate calculations, i.e. USD, EUR, RUB"
                        },
                        "date": {
                            "type": "string", 
                            "description": "A specific day to reference, in YYYY-MM-DD format."
                        },
                    },
                    "required": ["base_currency", "target_currency"],
                },
            },
        },
        # Tool 2 - Search Internet
        { 
            "type": "function",
            "function": {
                "name": "search_internet",
                "description": "Get internet search results for real time information",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The query to search the web for",
                        }
                    },
                    "required": ["search_query"],
                },
            },
        }
    ]

In [3]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [4]:
client = Groq(api_key=groq_api_key)

In [5]:
messages = [{"role": "user", 
             "content": "How much is a dollar worth in Japan? How about poland? Whats the current news in Argentina?"}]
response  = client.chat.completions.create(
                    messages=messages,
                    tools=first_tools,
                    model="llama3-70b-8192",
                    tool_choice="auto",  # auto is default, but we'll be explicit
)

In [6]:
# Function for printing out responses neatly
def pprint_response(response):
    print("--- Full Response ---\n")
    print(response, "\n")
    
    print("--- Chat Completion Message ---\n")
    print(response.choices[0].message, "\n")
    
    if response.choices[0].message.tool_calls:
        for i in range(0, len(response.choices[0].message.tool_calls)):
            print(f"--- Tool Call {i+1} ---\n")
            print(f"Function: {response.choices[0].message.tool_calls[i].function.name}\n")
            print(f"Arguments: {response.choices[0].message.tool_calls[i].function.arguments}\n")

In [7]:
pprint_response(response)

--- Full Response ---

ChatCompletion(choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChoiceMessage(content=None, role='assistant', tool_calls=[ChoiceMessageToolCall(id='call_b7ds', function=ChoiceMessageToolCallFunction(arguments='{"base_currency":"USD","target_currency":"JPY"}', name='get_exchange_rate'), type='function')]))], id='chatcmpl-22280619-c39d-456d-9d3b-735ad86efb4a', created=1721844405, model='llama3-70b-8192', object='chat.completion', system_fingerprint='fp_87cbfbbc4d', usage=Usage(completion_time=0.164454662, completion_tokens=53, prompt_time=0.108285509, prompt_tokens=1144, queue_time=None, total_time=0.27274017100000003, total_tokens=1197), x_groq={'id': 'req_01j3jys5aveh8akvhf934r80e5'}) 

--- Chat Completion Message ---

ChoiceMessage(content=None, role='assistant', tool_calls=[ChoiceMessageToolCall(id='call_b7ds', function=ChoiceMessageToolCallFunction(arguments='{"base_currency":"USD","target_currency":"JPY"}', name='get_exchange_rate')

In [8]:
client = Groq()
MODEL = 'llama3-70b-8192'

def calculate(expression):
    """Evaluate a mathematical expression"""
    try:
        result = eval(expression)
        return json.dumps({"result": result})
    except:
        return json.dumps({"error": "Invalid expression"})

def run_conversation(user_prompt):
    messages=[
        {
            "role": "system",
            "content": "You are a calculator assistant. Use the calculate function to perform mathematical operations and provide the results."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate",
                "description": "Evaluate a mathematical expression",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "The mathematical expression to evaluate",
                        }
                    },
                    "required": ["expression"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls:
        available_functions = {
            "calculate": calculate,
        }
        messages.append(response_message)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                expression=function_args.get("expression")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        return second_response.choices[0].message.content

user_prompt = "What is 25 * 4 + 10?"
print(run_conversation(user_prompt))

The result of the calculation 25 * 4 + 10 is indeed 110. 

Here's the step-by-step calculation:

1. 25 * 4 = 100
2. 100 + 10 = 110

So, the final answer is 110.


In [12]:
import inspect

# Dummy functions for demonstration
def get_exchange_rate(base_currency, target_currency, date=None):
    return f"Exchange rate from {base_currency} to {target_currency} on {date}"

def search_internet(search_query):
    return f"Search results for {search_query}"

# Main conversation function
def run_conversation(prompt, tools, tool_choice = "auto"):

    first_tools = [
        # Tool 1 - Get Exchange Rate
        { 
            "type": "function",
            "function": {
                "name": "get_exchange_rate",
                "description": "Get the current exchange rate of a base currency and target currency",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "base_currency": {
                            "type": "string",
                            "description": "The base currency for exchange rate calculations, i.e. USD, EUR, RUB",
                        },
                        "target_currency": {
                            "type": "string", 
                            "description": "The target currency for exchange rate calculations, i.e. USD, EUR, RUB"
                        },
                        "date": {
                            "type": "string", 
                            "description": "A specific day to reference, in YYYY-MM-DD format."
                        },
                    },
                    "required": ["base_currency", "target_currency"],
                },
            },
        },
        # Tool 2 - Search Internet
        { 
            "type": "function",
            "function": {
                "name": "search_internet",
                "description": "Get internet search results for real time information",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The query to search the web for",
                        }
                    },
                    "required": ["search_query"],
                },
            },
        }
    ]
    
    messages = [{"role": "user", "content": prompt}]
    
    print("\nInitial Message: ", messages)
    
    # Send the conversation and available functions to the model
    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=messages,
        tools=tools,
        tool_choice=tool_choice,
    )
    response_message = response.choices[0].message
    print("\nResponse Message: ", response_message)
    
    tool_calls = response_message.tool_calls
    print("\nTool Calls: ", tool_calls)
    
    # Check if the model wanted to call a function
    if tool_calls:
        
        # Call the functions
        available_functions = {
            "get_exchange_rate": get_exchange_rate,
            "search_internet": search_internet,
        } 
        # extend conversation with assistant's reply
        messages.append(response_message)
        
        # Call the function and add the response
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            
            
            # Get the function signature and call the function with given arguments
            sig = inspect.signature(function_to_call)
            call_args = {
                k: function_args.get(k, v.default)
                for k, v in sig.parameters.items()
                if k in function_args or v.default is not inspect.Parameter.empty
            }
            print(f"\nCalling {function_to_call} with arguments {call_args}")
            
            function_response = str(function_to_call(**call_args))
            
            print("\nFunction Response: ", function_response)

            # Put output into a tool message
            tool_message = {
                    "tool_call_id": tool_call.id, # Needed for Parallel Tool Calling
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            print("\nAppending Message: ", tool_message)
            
            # Extend conversation with function response
            messages.append(tool_message)  

        # Get a new response from the model where it can see the entire conversation including the function call outputs
        second_response = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=messages,
        )  

        print("\nLLM Response: ", second_response)

        print("\n---Formatted LLM Response---")
        print("\n",second_response.choices[0].message.content)
        
        return

prompt = "How much is a dollar worth in Japan? How about poland? Whats the current news in Argentina?"

run_conversation(prompt, first_tools)


Initial Message:  [{'role': 'user', 'content': 'How much is a dollar worth in Japan? How about poland? Whats the current news in Argentina?'}]

Response Message:  ChoiceMessage(content=None, role='assistant', tool_calls=[ChoiceMessageToolCall(id='call_pty8', function=ChoiceMessageToolCallFunction(arguments='{"base_currency":"USD","target_currency":"JPY"}', name='get_exchange_rate'), type='function'), ChoiceMessageToolCall(id='call_84kv', function=ChoiceMessageToolCallFunction(arguments='{"base_currency":"USD","target_currency":"PLN"}', name='get_exchange_rate'), type='function'), ChoiceMessageToolCall(id='call_jqf2', function=ChoiceMessageToolCallFunction(arguments='{"search_query":"Argentina news"}', name='search_internet'), type='function')])

Tool Calls:  [ChoiceMessageToolCall(id='call_pty8', function=ChoiceMessageToolCallFunction(arguments='{"base_currency":"USD","target_currency":"JPY"}', name='get_exchange_rate'), type='function'), ChoiceMessageToolCall(id='call_84kv', function

In [13]:
import json
import inspect

# Dummy functions for demonstration
def get_exchange_rate(base_currency, target_currency, date=None):
    return f"Exchange rate from {base_currency} to {target_currency} on {date}"

def search_internet(search_query):
    return f"Search results for {search_query}"

# Tool definitions
first_tools = [
    # Tool 1 - Get Exchange Rate
    { 
        "type": "function",
        "function": {
            "name": "get_exchange_rate",
            "description": "Get the current exchange rate of a base currency and target currency",
            "parameters": {
                "type": "object",
                "properties": {
                    "base_currency": {
                        "type": "string",
                        "description": "The base currency for exchange rate calculations, i.e. USD, EUR, RUB",
                    },
                    "target_currency": {
                        "type": "string", 
                        "description": "The target currency for exchange rate calculations, i.e. USD, EUR, RUB"
                    },
                    "date": {
                        "type": "string", 
                        "description": "A specific day to reference, in YYYY-MM-DD format."
                    },
                },
                "required": ["base_currency", "target_currency"],
            },
        },
    },
    # Tool 2 - Search Internet
    { 
        "type": "function",
        "function": {
            "name": "search_internet",
            "description": "Get internet search results for real time information",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_query": {
                        "type": "string",
                        "description": "The query to search the web for",
                    }
                },
                "required": ["search_query"],
            },
        },
    }
]

# Main conversation function
def run_conversation(prompt, tools, tool_choice="auto"):
    messages = [{"role": "user", "content": prompt}]
    
    print("\nInitial Message: ", messages)
    
    try:
        # Send the conversation and available functions to the model
        response = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        response_message = response.choices[0].message
        print("\nResponse Message: ", response_message)
        
        tool_calls = response_message.tool_calls
        print("\nTool Calls: ", tool_calls)
        
        # Check if the model wanted to call a function
        if tool_calls:
            # Call the functions
            available_functions = {
                "get_exchange_rate": get_exchange_rate,
                "search_internet": search_internet,
            }
            
            # Extend conversation with assistant's reply
            messages.append({"role": "assistant", "content": response_message.content})
            
            # Call the function and add the response
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                if not function_name:
                    continue
                function_to_call = available_functions.get(function_name)
                function_args = json.loads(tool_call.function.arguments)
                
                if not function_to_call:
                    print(f"\nFunction {function_name} not found in available functions.")
                    continue
                
                # Get the function signature and call the function with given arguments
                sig = inspect.signature(function_to_call)
                call_args = {
                    k: function_args.get(k, v.default)
                    for k, v in sig.parameters.items()
                    if k in function_args or v.default is not inspect.Parameter.empty
                }
                print(f"\nCalling {function_to_call} with arguments {call_args}")
                
                function_response = str(function_to_call(**call_args))
                
                print("\nFunction Response: ", function_response)

                # Put output into a tool message
                tool_message = {
                    "tool_call_id": tool_call.id, # Needed for Parallel Tool Calling
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
                print("\nAppending Message: ", tool_message)
                
                # Extend conversation with function response
                messages.append(tool_message)  

            # Get a new response from the model where it can see the entire conversation including the function call outputs
            second_response = client.chat.completions.create(
                model="llama3-70b-8192",
                messages=messages,
            )  

            print("\nLLM Response: ", second_response)

            print("\n---Formatted LLM Response---")
            print("\n", second_response.choices[0].message.content)
            
        else:
            print("\nNo tool calls were made by the model.")
            
    except Exception as e:
        print(f"An error occurred: {e}")

prompt = "How much is a dollar worth in Japan? How about Poland? What's the current news in Argentina?"

run_conversation(prompt, first_tools)



Initial Message:  [{'role': 'user', 'content': "How much is a dollar worth in Japan? How about Poland? What's the current news in Argentina?"}]

Response Message:  ChoiceMessage(content=None, role='assistant', tool_calls=[ChoiceMessageToolCall(id='call_61bp', function=ChoiceMessageToolCallFunction(arguments='{"base_currency":"USD","target_currency":"JPY"}', name='get_exchange_rate'), type='function'), ChoiceMessageToolCall(id='call_ex22', function=ChoiceMessageToolCallFunction(arguments='{"base_currency":"USD","target_currency":"PLN"}', name='get_exchange_rate'), type='function'), ChoiceMessageToolCall(id='call_wmks', function=ChoiceMessageToolCallFunction(arguments='{"search_query":"current news in Argentina"}', name='search_internet'), type='function')])

Tool Calls:  [ChoiceMessageToolCall(id='call_61bp', function=ChoiceMessageToolCallFunction(arguments='{"base_currency":"USD","target_currency":"JPY"}', name='get_exchange_rate'), type='function'), ChoiceMessageToolCall(id='call_ex2

In [14]:
from pydantic import BaseModel, Field, HttpUrl
from typing import List, Optional, Dict
from datetime import datetime

# Defining the tool properties with Pydantic
class SendEmailCampaign(BaseModel):
    recipients: List[str] = Field(
        ..., 
        description="List of strings, each an email address. Example: ['example1@mail.com', 'example2@mail.com']"
    )
    subject: str = Field(
        ..., 
        description="String specifying the email's subject line. Example: 'Exciting News!'"
    )
    body_text: str = Field(
        ..., 
        description="Plain text content of the email body. Example: 'We have some exciting updates to share with you.'"
    )
    attachments: Optional[List[HttpUrl]] = Field(
        default=[], 
        description="List of URLs to attachment files. Example: ['http://example.com/attachment1.pdf', 'http://example.com/attachment2.png']"
    )
    personalization: Optional[Dict[str, Dict[str, str]]] = Field(
        default={}, 
        description="Dictionary for personalizing email content. Key is recipient email, value is a dictionary of variables. Example: {'example1@mail.com': {'first_name': 'John'}, 'example2@mail.com': {'first_name': 'Jane'}}"
    )
    send_time: Optional[datetime] = Field(
        None, 
        description="The time when the email campaign is to be sent. Example: '2024-07-13T14:30:00'"
    )
    priority: Optional[str] = Field(
        default="normal", 
        description="Email priority level. Options: 'low', 'normal', 'high'. Example: 'high'", 
        enum=["low", "normal", "high"]
    )
    tracking: Optional[Dict[str, bool]] = Field(
        default={"open": True, "click": True}, 
        description="Tracking options for the email. Keys can be 'open' and 'click', values are booleans. Example: {'open': True, 'click': True}"
    )
    campaign_id: Optional[str] = Field(
        None, 
        description="Unique identifier for the email campaign for tracking purposes. Example: 'campaign_12345'"
    )

# Creating the Tool with PyDantic Schema
email_campaign_tool = [
    {
        "type": "function",
        "function": {
            "name": "send_email_campaign",
            "description": "Send out a marketing email campaign",
            "parameters": SendEmailCampaign.schema(), # Passing in our Pydantic schema
        },
    }
]

In [15]:
email_campaign_tool

[{'type': 'function',
  'function': {'name': 'send_email_campaign',
   'description': 'Send out a marketing email campaign',
   'parameters': {'properties': {'recipients': {'description': "List of strings, each an email address. Example: ['example1@mail.com', 'example2@mail.com']",
      'items': {'type': 'string'},
      'title': 'Recipients',
      'type': 'array'},
     'subject': {'description': "String specifying the email's subject line. Example: 'Exciting News!'",
      'title': 'Subject',
      'type': 'string'},
     'body_text': {'description': "Plain text content of the email body. Example: 'We have some exciting updates to share with you.'",
      'title': 'Body Text',
      'type': 'string'},
     'attachments': {'anyOf': [{'items': {'format': 'uri',
         'maxLength': 2083,
         'minLength': 1,
         'type': 'string'},
        'type': 'array'},
       {'type': 'null'}],
      'default': [],
      'description': "List of URLs to attachment files. Example: ['http:

In [19]:
prompt = """
"Can you send an email to adamlucek@youtube.com, samaltman@openai.com, and elonmusk@twitter.com about the cool youtube channel https://www.youtube.com/@AdamLucek 
for July 20th at 8 am pst, this is high priority and include just click tracking and name personalization, 
the campaign is campaign_13, 
also add my linkedin as an attachment https://www.linkedin.com/in/adamrlucek/,
Sign off your response with Adam Ł in the body.
"""

messages = [{"role": "user", 
             "content": prompt}]


response = client.chat.completions.create(
        model="llama3-groq-70b-8192-tool-use-preview",
        messages=messages,
        tools=email_campaign_tool,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )

In [20]:
pprint_response(response)

--- Full Response ---

ChatCompletion(choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChoiceMessage(content=None, role='assistant', tool_calls=[ChoiceMessageToolCall(id='call_wj71', function=ChoiceMessageToolCallFunction(arguments='{"recipients": ["adamlucek@youtube.com", "samaltman@openai.com", "elonmusk@twitter.com"], "subject": "Cool YouTube Channel", "body_text": "Check out this cool YouTube channel https://www.youtube.com/@AdamLucek. Best, Adam \\u0141", "campaign_id": "campaign_13", "attachments": ["https://www.linkedin.com/in/adamrlucek/"], "priority": "high", "send_time": "2023-07-20T08:00:00", "tracking": {"click": true}}', name='send_email_campaign'), type='function')]))], id='chatcmpl-dab7f291-d034-483f-bef3-f04c2856927d', created=1721845240, model='llama3-groq-70b-8192-tool-use-preview', object='chat.completion', system_fingerprint='fp_ee4b521143', usage=Usage(completion_time=0.466811903, completion_tokens=148, prompt_time=0.073171444, prompt_tok

In [21]:
# Example function of sending off an Email Campaign
def send_email_campaign(data: dict):
    try:
        # Validate and parse the data using the SendEmailCampaign model
        campaign = SendEmailCampaign(**data)
        
        # Simulate sending the email campaign
        print(f"\nSending email campaign '{campaign.campaign_id or 'N/A'}' with priority '{campaign.priority}':")
        for recipient in campaign.recipients:
            personalized_subject = campaign.subject
            if recipient in campaign.personalization:
                if 'first_name' in campaign.personalization[recipient]:
                    personalized_subject = f"{campaign.personalization[recipient]['first_name']}, {campaign.subject}"
            
            print("-----")
            print(f"\nTo: {recipient}")
            print(f"Subject: {personalized_subject}")
            print(f"Body: {campaign.body_text}")
            if campaign.attachments:
                attachment_urls = [str(url) for url in campaign.attachments]
                print(f"\tAttachments: {', '.join(attachment_urls)}")

        print("-----")
        print(f"\nScheduled Send Time: {campaign.send_time or 'Immediate'}")
        print(f"Campaign ID: {campaign.campaign_id or 'N/A'}")
        print(f"Tracking: {campaign.tracking}")
    except Exception as e:
        print(f"Failed to send email campaign: {e}")

args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
send_email_campaign(args)


Sending email campaign 'campaign_13' with priority 'high':
-----

To: adamlucek@youtube.com
Subject: Cool YouTube Channel
Body: Check out this cool YouTube channel https://www.youtube.com/@AdamLucek. Best, Adam Ł
	Attachments: https://www.linkedin.com/in/adamrlucek/
-----

To: samaltman@openai.com
Subject: Cool YouTube Channel
Body: Check out this cool YouTube channel https://www.youtube.com/@AdamLucek. Best, Adam Ł
	Attachments: https://www.linkedin.com/in/adamrlucek/
-----

To: elonmusk@twitter.com
Subject: Cool YouTube Channel
Body: Check out this cool YouTube channel https://www.youtube.com/@AdamLucek. Best, Adam Ł
	Attachments: https://www.linkedin.com/in/adamrlucek/
-----

Scheduled Send Time: 2023-07-20 08:00:00
Campaign ID: campaign_13
Tracking: {'click': True}


In [23]:
import os
from langchain_groq import ChatGroq

In [24]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [25]:
llm = ChatGroq(model="llama3-groq-70b-8192-tool-use-preview",temperature=0,max_tokens=4096)

In [28]:
from langchain_core.tools import tool
import requests

# Using the tool decorator to attach onto an existing function
@tool
def get_exchange_rate(base_currency: str, target_currency: str, date: str = "latest") -> float:
    """Get the latest exchange rate between two currency. Date defaults latest if not provided."""
    url = f"https://cdn.jsdelivr.net/npm/@fawazahmed0/currency-api@latest/v1/currencies/{base_currency.lower()}.json"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data.get(base_currency.lower(), {}).get(target_currency.lower(), None)
    else:
        raise Exception(f"Failed to fetch exchange rate: {response.status_code}")


# Using Pydantic schemas (non-invokable tool)
from langchain_core.pydantic_v1 import BaseModel, Field

class SendEmailCampaignLC(BaseModel):
    "Send a marketing email campaign out to your mailing list"
    
    recipients: List[str] = Field(
        ..., 
        description="List of strings, each an email address. Example: ['example1@mail.com', 'example2@mail.com']"
    )
    subject: str = Field(
        ..., 
        description="String specifying the email's subject line. Example: 'Exciting News!'"
    )
    body_text: str = Field(
        ..., 
        description="Plain text content of the email body. Example: 'We have some exciting updates to share with you.'"
    )

# Using built in LangChain tool integrations
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults

wrapper = DuckDuckGoSearchAPIWrapper(max_results=10)
web_search = DuckDuckGoSearchResults(api_wrapper=wrapper)

In [29]:
get_exchange_rate.invoke({"base_currency": "usd", "target_currency": "jpy"})

155.43789922

In [30]:
web_search.invoke("Fun things to do in SF")

"[snippet: If you want the best views of San Francisco, take a hike to Twin Peaks. These famous grassy peaks rise 922 feet in elevation, making them the second highest point in the city (after Mount Davidson ..., title: 30 Top-Rated Things to Do in San Francisco | U.S. News Travel, link: https://travel.usnews.com/San_Francisco_CA/Things_To_Do/], [snippet: The audio tour is one of the best things to do in San Francisco for tourists, but even locals will have plenty to learn on a visit to this historic island. 13. Ride a cable car. Photo by Ragnar Vorel on Unsplash. SF's cable cars are the only moving national landmark in the United States, dating back to 1873., title: 53 Best Attractions, Events, And Things To Do In San Francisco, link: https://secretsanfrancisco.com/things-to-do-san-francisco/], [snippet: The Atlas Obscura Guide To San Francisco 213 Cool, Hidden, and Unusual Things to Do in San Francisco, California Updated May 8, 2024 From the Gold Rush to Silicon Valley, the Bay Area

In [31]:
tools = [web_search, SendEmailCampaignLC, get_exchange_rate]
llm_tools = llm.bind_tools(tools)

In [35]:
# prompt = "How much is a dollar worth in Japan right now"
# prompt = "Can you send an email to adamlucek@youtube.com, samaltman@openai.com, and elonmusk@twitter.com about the cool youtube channel https://www.youtube.com/@AdamLucek"
# prompt = "When was langgraph cloud released?"
prompt = "When was langchain founded? and how much is a dollar worth in japan rn"

output = llm_tools.invoke(prompt)
print(f"output from the tool calling", output)
print("\n---Response---")
# print(output.content[0]['text'])
print("\n---Tool Calls---")
print(output.tool_calls)

output from the tool calling content='' additional_kwargs={'tool_calls': [{'id': 'call_pjpz', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "JPY"}', 'name': 'get_exchange_rate'}, 'type': 'function'}, {'id': 'call_2t2a', 'function': {'arguments': '{"query": "When was LangChain founded?"}', 'name': 'duckduckgo_results_json'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_time': 0.218846784, 'completion_tokens': 70, 'prompt_time': 0.042186621, 'prompt_tokens': 567, 'queue_time': None, 'total_time': 0.261033405, 'total_tokens': 637}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-500be2b7-917b-4a90-a40d-1e0e91d99560-0' tool_calls=[{'name': 'get_exchange_rate', 'args': {'base_currency': 'USD', 'target_currency': 'JPY'}, 'id': 'call_pjpz', 'type': 'tool_call'}, {'name': 'duckduckgo_results_json', 'args': {'query': 'When was LangChain founded?

In [36]:
from langchain_core.messages import HumanMessage, ToolMessage

query = "When was langchain founded? and how much is a dollar worth in japan rn"

messages = [HumanMessage(query)]
ai_msg = llm_tools.invoke(messages)
messages.append(ai_msg)

# Only using web search and exchange rate, and the Pydantic schema is not a full function, just a container for arguments
for tool_call in ai_msg.tool_calls:
    selected_tool = {"duckduckgo_results_json": web_search, "get_exchange_rate": get_exchange_rate}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

for i in range(0, len(messages)):
    print("-------")
    print(f"{messages[i].type}: ", messages[i])

-------
human:  content='When was langchain founded? and how much is a dollar worth in japan rn'
-------
ai:  content='' additional_kwargs={'tool_calls': [{'id': 'call_jk1p', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "JPY"}', 'name': 'get_exchange_rate'}, 'type': 'function'}, {'id': 'call_4yq7', 'function': {'arguments': '{"query": "When was LangChain founded?"}', 'name': 'duckduckgo_results_json'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_time': 0.220406694, 'completion_tokens': 70, 'prompt_time': 0.037153568, 'prompt_tokens': 567, 'queue_time': None, 'total_time': 0.257560262, 'total_tokens': 637}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-0a3b4c76-f407-40f8-b150-eec18190a12e-0' tool_calls=[{'name': 'get_exchange_rate', 'args': {'base_currency': 'USD', 'target_currency': 'JPY'}, 'id': 'call_jk1p', 'type': 'tool_call'}, {

In [40]:
# query = "When was langchain founded? and how much is a dollar worth in japan rn"

# messages = [HumanMessage(query)]
final_response = llm_tools.invoke(messages)
print(final_response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_htp4', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "JPY"}', 'name': 'get_exchange_rate'}, 'type': 'function'}, {'id': 'call_kjfm', 'function': {'arguments': '{"query": "When was LangChain founded?"}', 'name': 'duckduckgo_results_json'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_time': 0.219703729, 'completion_tokens': 70, 'prompt_time': 0.042037752, 'prompt_tokens': 567, 'queue_time': None, 'total_time': 0.261741481, 'total_tokens': 637}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-93d23973-1ed5-4b42-bf9e-518bafbebdaa-0' tool_calls=[{'name': 'get_exchange_rate', 'args': {'base_currency': 'USD', 'target_currency': 'JPY'}, 'id': 'call_htp4', 'type': 'tool_call'}, {'name': 'duckduckgo_results_json', 'args': {'query': 'When was LangChain founded?'}, 'id': 'call_kjfm', 'type'

In [41]:
from langchain import hub
from langchain_groq import ChatGroq
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# Get the prompt
oaif_prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatGroq(model="llama3-groq-70b-8192-tool-use-preview", temperature=0.7,max_tokens=4096 )
tools = [web_search, get_exchange_rate]

# Create the agent
oaif_agent = create_tool_calling_agent(llm, tools, oaif_prompt)

# Create the Agent Executor
# This is the runtime for an agent. This is what actually calls the agent, executes the actions it chooses, passes the action outputs back to the agent, and repeats. I
oaif_agent_executor = AgentExecutor(agent=oaif_agent, tools=tools, verbose=True)

In [45]:
query = "When was langgraph cloud released? and how much is a dollar worth in japan rn"
response = oaif_agent_executor.invoke({"input": query})
print(response)
# print("\n", response['output'][0])



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_results_json` with `{'query': 'When was LangGraph Cloud released?'}`


[snippet: Announcing LangGraph v0.1 & LangGraph Cloud: Running agents at scale, reliably. Our new infrastructure for running agents at scale, LangGraph Cloud, is available in beta. We also have a new stable release of LangGraph. At LangChain, we aim to make it easy to build LLM applications - systems that connect LLMs to external sources of data and ..., title: Announcing LangGraph v0.1 & LangGraph Cloud: Running agents at scale ..., link: https://blog.langchain.dev/langgraph-cloud/], [snippet: LangGraph. LangGraph is framework-agnostic, with each node functioning as a regular Python function. It extends the core Runnable API (a shared interface for streaming, async, and batch calls) to facilitate: Seamless state management across multiple conversation turns or tool calls. Flexible routing between nodes based on dynamic criteria., title: LangChain Just L

In [48]:
from langchain.agents import create_react_agent

react_prompt = hub.pull("hwchase17/react")
react_agent = create_react_agent(llm, tools, react_prompt)
react_agent_executor = AgentExecutor(agent=react_agent, tools=tools, verbose=True,handle_parsing_errors=True)

In [49]:
query = "When was langgraph cloud released? When was LangChain founded?"
response = react_agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
First, I need to find out when LangGraph Cloud was released. I'll use DuckDuckGo for this.

Action: duckduckgo_results_json
Action Input: When was LangGraph Cloud released?[snippet: Announcing LangGraph v0.1 & LangGraph Cloud: Running agents at scale, reliably. Our new infrastructure for running agents at scale, LangGraph Cloud, is available in beta. We also have a new stable release of LangGraph. At LangChain, we aim to make it easy to build LLM applications - systems that connect LLMs to external sources of data and ..., title: Announcing LangGraph v0.1 & LangGraph Cloud: Running agents at scale ..., link: https://blog.langchain.dev/langgraph-cloud/], [snippet: LangChain has unveiled two major developments aimed at enhancing the deployment and management of AI agents. The company announced the stable release of LangGraph v0.1 and introduced LangGraph Cloud, an infrastructure designed to run agents at scale, according to the LangChain Blog.. Lan

In [50]:
oaif_prompt = hub.pull("hwchase17/openai-functions-agent")

In [51]:
oaif_prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplat